<a href="https://colab.research.google.com/github/chunbo777/dacon_prac/blob/main/%EC%98%81%ED%99%94%EB%B6%84%EC%84%9D2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 데이터 및 라이브러리 로딩

In [69]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

train = pd.read_csv('/content/drive/MyDrive/dacon_practice/영화 관객수/movies_train.csv')
test = pd.read_csv('/content/drive/MyDrive/dacon_practice/영화 관객수/movies_test.csv')

## 데이터탐색

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           600 non-null    object 
 1   distributor     600 non-null    object 
 2   genre           600 non-null    object 
 3   release_time    600 non-null    object 
 4   time            600 non-null    int64  
 5   screening_rat   600 non-null    object 
 6   director        600 non-null    object 
 7   dir_prev_bfnum  270 non-null    float64
 8   dir_prev_num    600 non-null    int64  
 9   num_staff       600 non-null    int64  
 10  num_actor       600 non-null    int64  
 11  box_off_num     600 non-null    int64  
dtypes: float64(1), int64(5), object(6)
memory usage: 56.4+ KB


In [9]:
train.head()

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num
0,개들의 전쟁,롯데엔터테인먼트,액션,2012-11-22,96,청소년 관람불가,조병옥,NaN,0,91,2,23398
1,내부자들,(주)쇼박스,느와르,2015-11-19,130,청소년 관람불가,우민호,1161602.50,2,387,3,7072501
2,은밀하게 위대하게,(주)쇼박스,액션,2013-06-05,123,15세 관람가,장철수,220775.25,4,343,4,6959083
3,나는 공무원이다,(주)NEW,코미디,2012-07-12,101,전체 관람가,구자홍,23894.00,2,20,6,217866
4,불량남녀,쇼박스(주)미디어플렉스,코미디,2010-11-04,108,15세 관람가,신근호,1.00,1,251,2,483387


In [42]:
test.head()

,time,dir_prev_num,num_staff,num_actor,년,월,distributor_(주)NEW,distributor_(주)마운틴픽쳐스,distributor_CJ 엔터테인먼트,distributor_기타,distributor_롯데엔터테인먼트,distributor_인디스토리,genre_SF,genre_공포,genre_느와르,genre_다큐멘터리,genre_드라마,genre_멜로/로맨스,genre_뮤지컬,genre_미스터리,genre_서스펜스,genre_애니메이션,genre_액션,genre_코미디,screening_rat_12세 관람가,screening_rat_15세 관람가,screening_rat_전체 관람가,screening_rat_청소년 관람불가
0,125,2,304,3,2010,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
1,113,4,275,3,2010,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
2,115,3,419,7,2010,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
3,116,2,408,2,2010,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
4,110,1,380,1,2010,2,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0


## 데이터 전처리

In [70]:
# def distributor_mapping(train):
group_dis = train.groupby(["distributor"] , as_index = False)["box_off_num"].mean()
group_dis = group_dis.sort_values(by = "box_off_num", ascending=False)

In [71]:
group_direc = train.groupby(["director"] , as_index = False)["box_off_num"].mean()
group_direc = group_direc.sort_values(by = "box_off_num", ascending=False)

In [72]:
group_direc["box_off_num"] =group_direc["box_off_num"].apply(lambda x : int(x))

In [73]:
group_dis["box_off_num"] =group_dis["box_off_num"].apply(lambda x : int(x))

In [74]:
group_dis.set_index("distributor", inplace = True)

In [75]:
group_direc.set_index("director", inplace = True)

In [76]:
group_direc[:10]

,box_off_num
director,
윤제균,14262766
최동훈,12845252
양우석,11374879
봉준호,9350351
한재림,9135806
황동혁,8659725
이석훈,8212959
김한민,7482180
추창민,6986501


In [77]:
train["test_is"] = False
test["test_is"] = True

In [78]:
train_test = pd.concat([train, test], ignore_index=True)

In [79]:
train_test.head()

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num,test_is
0,개들의 전쟁,롯데엔터테인먼트,액션,2012-11-22,96,청소년 관람불가,조병옥,NaN,0,91,2,23398.0,False
1,내부자들,(주)쇼박스,느와르,2015-11-19,130,청소년 관람불가,우민호,1161602.50,2,387,3,7072501.0,False
2,은밀하게 위대하게,(주)쇼박스,액션,2013-06-05,123,15세 관람가,장철수,220775.25,4,343,4,6959083.0,False
3,나는 공무원이다,(주)NEW,코미디,2012-07-12,101,전체 관람가,구자홍,23894.00,2,20,6,217866.0,False
4,불량남녀,쇼박스(주)미디어플렉스,코미디,2010-11-04,108,15세 관람가,신근호,1.00,1,251,2,483387.0,False


In [80]:
group_dis.index

Index(['CJ E&M Pictures', '(주)쇼박스', '(주)아이필름코퍼레이션', '쇼박스(주)미디어플렉스', '영구아트무비',
       'CJ E&M 영화부문', '(주)NEW', 'CJ 엔터테인먼트', '이십세기폭스코리아(주)', 'SK텔레콤(주)',
       ...
       '건시네마', '(주)씨엠닉스', '위더스필름', '인터콘미디어', '한국YWCA연합회', '나우콘텐츠', '위드시네마',
       '사람과 사람들', '고구마공작소', '인피니티엔터테인먼트'],
      dtype='object', name='distributor', length=169)

In [81]:
group_dis.index[1]

'(주)쇼박스'

In [82]:
list(enumerate(train_test["distributor"].values))[0][1] in group_dis.index

True

In [89]:
group_direc.box_off_num.mean()

667891.1144067796

In [90]:
train_test["direc"] = 0
for i, director in enumerate(train_test["director"].values):
    if director in group_direc.index:
        train_test["direc"][i] = group_direc.loc[director].box_off_num
    else:
        train_test["direc"][i] = group_direc.box_off_num.mean()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [91]:
train_test

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num,test_is,direc
0,개들의 전쟁,롯데엔터테인먼트,액션,2012-11-22,96,청소년 관람불가,조병옥,NaN,0,91,2,23398.0,False,23398
1,내부자들,(주)쇼박스,느와르,2015-11-19,130,청소년 관람불가,우민호,1161602.50,2,387,3,7072501.0,False,2870137
2,은밀하게 위대하게,(주)쇼박스,액션,2013-06-05,123,15세 관람가,장철수,220775.25,4,343,4,6959083.0,False,6959083
3,나는 공무원이다,(주)NEW,코미디,2012-07-12,101,전체 관람가,구자홍,23894.00,2,20,6,217866.0,False,217866
4,불량남녀,쇼박스(주)미디어플렉스,코미디,2010-11-04,108,15세 관람가,신근호,1.00,1,251,2,483387.0,False,483387
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
838,해에게서 소년에게,디씨드,드라마,2015-11-19,78,15세 관람가,안슬기,2590.00,1,4,4,NaN,True,667891
839,울보 권투부,인디스토리,다큐멘터리,2015-10-29,86,12세 관람가,이일하,NaN,0,18,2,NaN,True,667891
840,어떤살인,(주)컨텐츠온미디어,느와르,2015-10-28,107,청소년 관람불가,안용훈,NaN,0,224,4,NaN,True,667891
841,말하지 못한 비밀,(주)씨타마운틴픽쳐스,드라마,2015-10-22,102,청소년 관람불가,송동윤,50699.00,1,68,7,NaN,True,667891


In [92]:
train_test["dis"] = 0
for i, distributor in enumerate(train_test["distributor"].values):
    if distributor in group_dis.index:
        train_test["dis"][i] = group_dis.loc[distributor].box_off_num
    else:
        train_test["dis"][i] = group_dis.box_off_num.mean()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [121]:
train_test["dis"]

0      1238148
1      3386656
2      3386656
3      2184404
4      2634823
        ...   
838          0
839       5210
840          0
841          0
842       1027
Name: dis, Length: 843, dtype: int64

In [93]:
train_test

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num,test_is,direc,dis
0,개들의 전쟁,롯데엔터테인먼트,액션,2012-11-22,96,청소년 관람불가,조병옥,NaN,0,91,2,23398.0,False,23398,1238148
1,내부자들,(주)쇼박스,느와르,2015-11-19,130,청소년 관람불가,우민호,1161602.50,2,387,3,7072501.0,False,2870137,3386656
2,은밀하게 위대하게,(주)쇼박스,액션,2013-06-05,123,15세 관람가,장철수,220775.25,4,343,4,6959083.0,False,6959083,3386656
3,나는 공무원이다,(주)NEW,코미디,2012-07-12,101,전체 관람가,구자홍,23894.00,2,20,6,217866.0,False,217866,2184404
4,불량남녀,쇼박스(주)미디어플렉스,코미디,2010-11-04,108,15세 관람가,신근호,1.00,1,251,2,483387.0,False,483387,2634823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
838,해에게서 소년에게,디씨드,드라마,2015-11-19,78,15세 관람가,안슬기,2590.00,1,4,4,NaN,True,667891,261436
839,울보 권투부,인디스토리,다큐멘터리,2015-10-29,86,12세 관람가,이일하,NaN,0,18,2,NaN,True,667891,5210
840,어떤살인,(주)컨텐츠온미디어,느와르,2015-10-28,107,청소년 관람불가,안용훈,NaN,0,224,4,NaN,True,667891,261436
841,말하지 못한 비밀,(주)씨타마운틴픽쳐스,드라마,2015-10-22,102,청소년 관람불가,송동윤,50699.00,1,68,7,NaN,True,667891,261436


In [94]:
# 결측치가 많은 데이터 제거
train = train.drop(['dir_prev_bfnum'],axis = 1)
test =  test.drop(['dir_prev_bfnum'],axis = 1)

In [95]:
train_test.drop(["dir_prev_bfnum"], axis = 1, inplace = True)

In [96]:
# 감독명 : 너무 다양해서 제거
train = train.drop(['director'],axis = 1)
test = test.drop(['director'],axis = 1)

In [97]:
train_test = train_test.drop(['director'],axis = 1)

In [98]:

# 제목 : 의미가 없기 때문에 제거
train = train.drop(['title'],axis= 1)
test = test.drop(['title'],axis= 1)

In [99]:
train_test = train_test.drop(['title'],axis= 1)

In [100]:
train.distributor.value_counts()

CJ 엔터테인먼트     54
롯데엔터테인먼트      52
(주)NEW        30
(주)마운틴픽쳐스     29
인디스토리         26
              ..
OAL(올)         1
(주)콘텐츠 윙       1
판씨네마(주)        1
제나두 엔터테인먼트     1
하준사            1
Name: distributor, Length: 169, dtype: int64

In [101]:
# 개봉일을 바탕으로 년,월 변수 생성
train_test['년'] = train_test['release_time'].apply(lambda x: int(x[:4]))
train_test['월'] = train_test['release_time'].apply(lambda x: int(x[5:7]))
# train_test =  train_test.drop(['release_time'],axis = 1)

# test['년'] = test['release_time'].apply(lambda x: int(x[:4]))
# test['월'] = test['release_time'].apply(lambda x: int(x[5:7]))
# test =  test.drop(['release_time'],axis = 1)


In [102]:
train_test =  train_test.drop(['release_time'],axis = 1)

In [103]:
train_test.drop(["distributor"], axis=1, inplace = True)

In [104]:
train_test

,genre,time,screening_rat,dir_prev_num,num_staff,num_actor,box_off_num,test_is,direc,dis,년,월
0,액션,96,청소년 관람불가,0,91,2,23398.0,False,23398,1238148,2012,11
1,느와르,130,청소년 관람불가,2,387,3,7072501.0,False,2870137,3386656,2015,11
2,액션,123,15세 관람가,4,343,4,6959083.0,False,6959083,3386656,2013,6
3,코미디,101,전체 관람가,2,20,6,217866.0,False,217866,2184404,2012,7
4,코미디,108,15세 관람가,1,251,2,483387.0,False,483387,2634823,2010,11
...,...,...,...,...,...,...,...,...,...,...,...,...
838,드라마,78,15세 관람가,1,4,4,NaN,True,667891,261436,2015,11
839,다큐멘터리,86,12세 관람가,0,18,2,NaN,True,667891,5210,2015,10
840,느와르,107,청소년 관람불가,0,224,4,NaN,True,667891,261436,2015,10
841,드라마,102,청소년 관람불가,1,68,7,NaN,True,667891,261436,2015,10


In [105]:
# 원핫 인코딩
train_test_genre = pd.get_dummies(train_test["genre"])
# test = pd.get_dummies(test)

In [106]:
train_test_scrat = pd.get_dummies(train_test["screening_rat"])

In [107]:
train_test = pd.concat([train_test, train_test_genre, train_test_scrat], axis = 1)

In [108]:
train_test

,genre,time,screening_rat,dir_prev_num,num_staff,num_actor,box_off_num,test_is,direc,dis,년,월,SF,공포,느와르,다큐멘터리,드라마,멜로/로맨스,뮤지컬,미스터리,서스펜스,애니메이션,액션,코미디,12세 관람가,15세 관람가,전체 관람가,청소년 관람불가
0,액션,96,청소년 관람불가,0,91,2,23398.0,False,23398,1238148,2012,11,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
1,느와르,130,청소년 관람불가,2,387,3,7072501.0,False,2870137,3386656,2015,11,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
2,액션,123,15세 관람가,4,343,4,6959083.0,False,6959083,3386656,2013,6,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
3,코미디,101,전체 관람가,2,20,6,217866.0,False,217866,2184404,2012,7,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
4,코미디,108,15세 관람가,1,251,2,483387.0,False,483387,2634823,2010,11,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
838,드라마,78,15세 관람가,1,4,4,NaN,True,667891,261436,2015,11,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
839,다큐멘터리,86,12세 관람가,0,18,2,NaN,True,667891,5210,2015,10,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
840,느와르,107,청소년 관람불가,0,224,4,NaN,True,667891,261436,2015,10,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
841,드라마,102,청소년 관람불가,1,68,7,NaN,True,667891,261436,2015,10,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1


In [109]:
train_test.drop(["genre", "screening_rat"], axis = 1, inplace = True)

In [110]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
cols_to_scale = ["time", "num_staff", "direc", "dis"]
scaled_cols = scaler.fit_transform(train_test[cols_to_scale])
train_test[cols_to_scale] = scaled_cols
train_test.head()
 

,time,dir_prev_num,num_staff,num_actor,box_off_num,test_is,direc,dis,년,월,SF,공포,느와르,다큐멘터리,드라마,멜로/로맨스,뮤지컬,미스터리,서스펜스,애니메이션,액션,코미디,12세 관람가,15세 관람가,전체 관람가,청소년 관람불가
0,-0.108917,0,-0.379286,2,23398.0,False,-0.451966,0.505947,2012,11,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
1,0.388896,2,1.417543,3,7072501.0,False,1.347601,2.632249,2015,11,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
2,0.286405,4,1.150446,4,6959083.0,False,3.932429,2.632249,2013,6,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
3,-0.035709,2,-0.810283,6,217866.0,False,-0.329033,1.442423,2012,7,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
4,0.066782,1,0.591973,2,483387.0,False,-0.161184,1.888187,2010,11,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0


In [111]:
train_test

,time,dir_prev_num,num_staff,num_actor,box_off_num,test_is,direc,dis,년,월,SF,공포,느와르,다큐멘터리,드라마,멜로/로맨스,뮤지컬,미스터리,서스펜스,애니메이션,액션,코미디,12세 관람가,15세 관람가,전체 관람가,청소년 관람불가
0,-0.108917,0,-0.379286,2,23398.0,False,-0.451966,0.505947,2012,11,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
1,0.388896,2,1.417543,3,7072501.0,False,1.347601,2.632249,2015,11,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
2,0.286405,4,1.150446,4,6959083.0,False,3.932429,2.632249,2013,6,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
3,-0.035709,2,-0.810283,6,217866.0,False,-0.329033,1.442423,2012,7,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
4,0.066782,1,0.591973,2,483387.0,False,-0.161184,1.888187,2010,11,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
838,-0.372466,1,-0.907408,4,NaN,True,-0.044550,-0.460671,2015,11,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
839,-0.255333,0,-0.822423,2,NaN,True,-0.044550,-0.714249,2015,10,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
840,0.052140,0,0.428073,4,NaN,True,-0.044550,-0.460671,2015,10,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
841,-0.021068,1,-0.518905,7,NaN,True,-0.044550,-0.460671,2015,10,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1


In [112]:
train_df = train_test[train_test["test_is"]==False].drop("test_is", axis =1)

In [113]:
test_df = train_test[train_test["test_is"]==True].drop("test_is", axis =1)

In [114]:
test_df.drop("box_off_num", axis=1, inplace = True)

In [115]:
test_df.head()

,time,dir_prev_num,num_staff,num_actor,direc,dis,년,월,SF,공포,느와르,다큐멘터리,드라마,멜로/로맨스,뮤지컬,미스터리,서스펜스,애니메이션,액션,코미디,12세 관람가,15세 관람가,전체 관람가,청소년 관람불가
600,0.315688,2,0.913702,3,-0.044550,0.168169,2010,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
601,0.139989,4,0.737661,3,-0.044550,2.632249,2010,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
602,0.169273,3,1.611794,7,-0.044550,1.420985,2010,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
603,0.183914,2,1.545020,2,1.395021,2.632249,2010,2,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
604,0.096065,1,1.375050,1,-0.044550,1.420985,2010,2,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0


## 모델정의 및 학습

In [116]:
train_x = train_df.drop(['box_off_num'],axis= 1)
train_y = train_df['box_off_num']

In [117]:
model=RandomForestRegressor(n_estimators=100)
model.fit(train_x,train_y)

RandomForestRegressor()

# 학습 된 모델로 예측 데이터 생성

In [118]:
pred = model.predict(test_df)

## 제출파일 생성

In [119]:
submission = pd.read_csv('/content/drive/MyDrive/dacon_practice/영화 관객수/submission.csv')
submission

,title,box_off_num
0,용서는 없다,0
1,아빠가 여자를 좋아해,0
2,하모니,0
3,의형제,0
4,평행 이론,0
...,...,...
238,해에게서 소년에게,0
239,울보 권투부,0
240,어떤살인,0
241,말하지 못한 비밀,0


In [120]:
submission['box_off_num'] = pred

In [121]:
submission

,title,box_off_num
0,용서는 없다,692229.71
1,아빠가 여자를 좋아해,756723.75
2,하모니,761038.02
3,의형제,3654907.24
4,평행 이론,721859.42
...,...,...
238,해에게서 소년에게,251094.36
239,울보 권투부,58709.42
240,어떤살인,263767.49
241,말하지 못한 비밀,281059.08


In [122]:
submission.to_csv('베이스라인_2.csv',index = False)